In [1]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


### Select Object Indices of Interest

| Object Indices |[]()|[]()|
|---|---|---|
| __1.__ 002_master_chef_can | __8.__ 009_gelatin_box      | __15.__ 035_power_drill       |
| __2.__ 003_cracker_box     | __9.__ 010_potted_meat_can  | __16.__ 036_wood_block        |
| __3.__ 004_sugar_box       | __10.__ 011_banana          | __17.__ 037_scissors          |
| __4.__ 005_tomato_soup_can | __11.__ 019_pitcher_base    | __18.__ 040_large_marker      |
| __5.__ 006_mustard_bottle  | __12.__ 021_bleach_cleanser | __19.__ 051_large_clamp       |
| __6.__ 007_tuna_fish_can   | __13.__ 024_bowl            | __20.__ 052_extra_large_clamp |
| __7.__ 008_pudding_box     | __14.__ 025_mug             | __21.__ 061_foam_brick        |

In [2]:
from object_pose_utils.utils.pose_error import accuracyAUC
dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'

non_sym_objs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18,]
sym_objs = [13, 16, 19, 20, 21]
textureless_objects = set([10, 11, 13, 14, 16, 19, 20, 21])
textured_objects = set(range(1,22)) - set([10, 11, 13, 14, 16, 19, 20, 21])


with open('{0}/image_sets/classes.txt'.format(dataset_root)) as f:                                    
    classes = f.read().split()
classes.insert(0, '__background__')

def non_func(x):
    return x

def mean_abs(x):
    return np.mean(np.abs(x))

def log_clean(x):
    return np.log(np.maximum(1e-6, x))

def mean_log_clean(x):
    return np.mean(log_clean(x))

def mean_log(x):
    return np.mean(np.log(x))   

def accuracyAUC100(x, max_theshold):
    return accuracyAUC(x, max_theshold)*100

def median_log(x):
    return np.median(np.log(x))

def makeTableEntries(data_dict, keys = None, objs = list(range(1,22)), 
              val_func = non_func, bold_func = max,
              individual = False,
              agg_title = 'All'):
    if(keys is None):
        keys = list(data_dict.keys())
    
    data_str = ''
    if(individual):
        for obj in objs:    
            data_str += ' '.join(classes[obj].split('_')[1:]) + '\n'
            vals = [val_func(np.array(data_dict[k][obj])) for k in keys]
            bold_val = bold_func(vals)
            for v in vals:
                if(v == bold_val):
                    data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
                else:
                    data_str += ' & {:0.2f}'.format(v)
            data_str += ' \\\\'
            data_str += '\n'

        data_str += '\hline\n'

    data_str += '{} \n'.format(agg_title)
    vals = [val_func(np.concatenate([np.array(data_dict[k][o]) for o in objs])) for k in keys]
    bold_val = bold_func(vals)
    for v in vals:
        if(v == bold_val):
            data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
        else:
            data_str += ' & {:0.2f}'.format(v)
    data_str += ' \\\\'
    data_str += '\n'
    #print(data_str)
    return data_str

In [3]:
from transforms3d.quaternions import quat2mat, mat2quat

def getPoseCNNQuat(data, obj):
    pose_idx = np.where(data['rois'][:,1].flatten()==obj)[0]
    if(len(pose_idx) == 0):
        return None
    else:
        pose_idx = pose_idx[0]
    pose = data['poses'][pose_idx]
    q = pose[:4][[1,2,3,0]]
    q /= np.linalg.norm(q)
    t = pose[4:7]
    return q

## YCB Dataset

In [15]:
from object_pose_utils.datasets.linemod_dataset import LinemodDataset
from object_pose_utils.datasets.image_processing import ImageNormalizer
from object_pose_utils.datasets.sixdc_dataset import ply_vtx

from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes


dataset_root = "/ssd0/datasets/linemod/Linemod_preprocessed/"
object_list = [1,2,4,5,6,8,9,10,11,12,13,14,15]

#object_id = 1

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.IMAGE_CROPPED,
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES,
                ]

dataset = LinemodDataset(dataset_root, 
                         mode = 'eval',
                         object_list = object_list,#[object_id], 
                         output_data = output_format,
                         add_syn_noise = False,
                         add_syn_background = False,
                         resample_on_error = False,
                         #preprocessors = [InplaneRotator(0)],
                         postprocessors = [ImageNormalizer()],
                         image_size = [640, 480], 
                         num_points=500,
                         segnet_mask = True)


model_clouds = {}
for j, object_id in enumerate(object_list):
    model_filename = '{}/models/obj_{:02d}.ply'.format(dataset_root, object_id)

    model_cloud = ply_vtx(model_filename)/1000.
    model_cloud = model_cloud[np.random.choice(len(model_cloud), 1000, replace=False)]
    
    model_clouds[j] = model_cloud

## Pose Estimator

In [5]:
from dense_fusion.network import PoseNet

model_checkpoint = '/home/bokorn/src/DenseFusion/trained_checkpoints/linemod/pose_model_9_0.01310166542980859.pth'

num_objects = 13 #number of object classes in the dataset
num_points = 500 #number of points on the input pointcloud
df_estimator = PoseNet(num_points = num_points, 
                    num_obj = num_objects)
df_estimator.load_state_dict(torch.load(model_checkpoint))
df_estimator.cuda();

In [24]:
df_results_filename = 'results_linemod_test_df_pose_model_9_0.01310166542980859_pose_refine_model_493_0.006761023565178073.pkl'

with open(df_results_filename, 'rb') as f:
    df_results = pickle.load(f)
    

## Distribution Estimators

In [6]:
ls /scratch/bokorn/results/log_lik_linemod/df_global_comp/

lr_1e-5/


In [44]:
from se3_distributions.losses.loglik_loss import evaluateFeature
from se3_distributions.models.compare_networks import SigmoidCompareNet, SigmoidNet

#feature_root = '/scratch/bokorn/results/dense_fusion_global_feat/'
feature_root = '/scratch/datasets/linemod/'
feature_key = 'feat_global'
feature_size = 1024

grid_vertices = torch.load(os.path.join(feature_root, 'grid',
    '{}_vertices.pt'.format(dataset.classes[1])))

grid_features = {}
for j, object_id in enumerate(object_list):
    grid_features[j+1] = torch.load(os.path.join(feature_root, 'grid',
        '{}_{}_features.pt'.format(feature_key, dataset.classes[object_id])))

comp_model_checkpoint = '/scratch/bokorn/results/log_lik_linemod/df_global_comp/lr_1e-5/2020-02-28_20-38-07/weights/checkpoint_91000.pth'

comp_estimator = SigmoidCompareNet(feature_size, len(object_list))
comp_estimator.load_state_dict(torch.load(comp_model_checkpoint))
comp_estimator.cuda();
comp_estimator.eval();

In [46]:
feature_root = '/scratch/datasets/ycb/'
feature_key = 'feat_global'
feature_size = 1024
grid_size = 3885

reg_model_checkpoint = '/scratch/bokorn/results/log_lik_linemod/df_global_reg/lr_1e-5/2020-02-28_21-13-23/weights/checkpoint_197000.pth'

reg_estimator = SigmoidNet(feature_size, len(object_list)*grid_size)
reg_estimator.load_state_dict(torch.load(reg_model_checkpoint))
reg_estimator.cuda();
reg_estimator.eval();

In [47]:
if(False):
    confusion_root = '/home/bokorn/data/confusion_matrices'
    confusion_eps = 0.000001

    confusion_matrices = {}
    for object_id in object_list:
        confusion_matrices[object_id] = scio.loadmat(os.path.join(confusion_root, 
            "{0}_confusion_matrix.mat".format(object_id)))['loaded'] + confusion_eps

In [48]:
from object_pose_utils.utils.bingham import iso_loss_calculation, duel_loss_calculation

class isoLikelihood(object):
        def __init__(self, mean_q, sig):
            self.mean_q = mean_q
            self.sig = sig.flatten()
            
        def __call__(self, quats):
            likelihoods = []
            for q in quats.unsqueeze(1):
                loss, lik = iso_loss_calculation(self.mean_q, torch.abs(self.sig), q)
                likelihoods.append(lik*2.0)
            return torch.stack(likelihoods)
        
class duelLikelihood(object):
        def __init__(self, mean_q, duel_q, z):
            self.mean_q = mean_q
            self.duel_q = duel_q
            self.z = z.flatten()
            
        def __call__(self, quats):
            likelihoods = []
            for q in quats.unsqueeze(1):
                loss, lik = duel_loss_calculation(self.mean_q, self.duel_q, 
                                                  -torch.abs(self.z), q)
                likelihoods.append(lik*2.0)
            return torch.stack(likelihoods)


In [49]:
from se3_distributions.models.bingham_networks import IsoBingham
from se3_distributions.losses.bingham_loss import isoLikelihood

if(True):
    feature_size = 1024
    #feature_size = 1408
    #iso_model_checkpoint_pattern = '/scratch/bokorn/results/log_lik/df_local_full_orig_iso/**/weights/checkpoint_*.pth'
    #iso_model_checkpoint = sorted(glob.glob(iso_model_checkpoint_pattern,
    #                                        recursive=True))[-1]
    iso_model_checkpoint = '/scratch/bokorn/results/log_lik_linemod/df_global_iso/lr_1e-5/2020-02-28_21-36-08/weights/best_quat.pth'
    print(glob.glob('/'.join(iso_model_checkpoint.split('/')[:-1]) + '/*', recursive=True))
    iso_estimator = IsoBingham(feature_size, len(object_list))
    iso_estimator.load_state_dict(torch.load(iso_model_checkpoint))
    iso_estimator.eval()
    iso_estimator.cuda()

    def bing_iso(res, obj):
        max_q, max_t, feat = res['max_q'], res['max_t'], res['global_feat']
        
        #feat = torch.Tensor(feat).unsqueeze(0).cuda()
        mean_est = torch.Tensor(max_q).unsqueeze(0).cuda()
        df_obj = torch.LongTensor(obj-1).unsqueeze(0).cuda()
        sig_est = iso_estimator(feat.unsqueeze(0).cuda(), df_obj)
        lik_est = isoLikelihood(mean_q=mean_est[0], 
                                sig=sig_est[0,0])
        
        return lik_est, max_q, max_t

['/scratch/bokorn/results/log_lik_linemod/df_global_iso/lr_1e-5/2020-02-28_21-36-08/weights/checkpoint_1751000.pth', '/scratch/bokorn/results/log_lik_linemod/df_global_iso/lr_1e-5/2020-02-28_21-36-08/weights/best_quat.pth']


In [50]:
from object_pose_utils.utils.bingham import isobingham_likelihood

sigmas_data = np.load('/home/bokorn/src/DenseFusion/df_orig_sigmas.npz', allow_pickle=True)
sigmas_single = sigmas_data['single_sigma'].item()
sigmas_mixture = sigmas_data['mixture_sigma'].item()

In [51]:
from object_pose_utils.utils.pose_processing import quatAngularDiffBatch
from object_pose_utils.utils.interpolation import BinghamInterpolation, TetraInterpolation
from se3_distributions.utils.evaluation_utils import evaluateDenseFusion

def hist_reg_global(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj)
    lik_est = evaluateFeature(reg_estimator, obj, feat, None)
    lik_est = to_np(lik_est.flatten())
    lik_est /= lik_est.sum()
    return lik_est, max_q, max_t

def hist_reg_idv_global(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj)
    lik_est = evaluateFeature(reg_estimator_idv[obj.item()], obj, feat, None)
    lik_est = to_np(lik_est.flatten())
    lik_est /= lik_est.sum()
    return lik_est, max_q, max_t

def hist_comp_global(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj)
    lik_est = evaluateFeature(comp_estimator, obj, feat, grid_features)
    lik_est = to_np(lik_est.flatten())
    lik_est /= lik_est.sum()
    return lik_est, max_q, max_t

def hist_comp_local(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj, use_global_feat=False)
    lik_est = evaluateFeature(comp_estimator_local, obj, feat, grid_features_local)
    lik_est = to_np(lik_est.flatten())
    lik_est /= lik_est.sum()
    return lik_est, max_q, max_t

def hist_comp_global_prop(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj)
    lik_est = evaluateFeature(prop_comp_estimator, obj, feat, grid_features)
    lik_est = to_np(lik_est.flatten())
    lik_est /= lik_est.sum()
    return lik_est, max_q, max_t

def hist_comp_global_funnel(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj)
    lik_est = evaluateFeature(funnel_comp_estimator, obj, feat, grid_features)
    lik_est = to_np(lik_est.flatten())
    lik_est /= lik_est.sum()
    return lik_est, max_q, max_t

def hist_conf(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj)
    dists = quatAngularDiffBatch(max_q, to_np(grid_vertices))
    bin_idx = np.argmin(dists)
    lik_est = confusion_matrices[obj.item()][bin_idx]
    lik_est /= lik_est.sum()
    return lik_est, max_q, max_t

def bing_fixed(img, points, choose, obj):
    max_q, max_t, feat = evaluateDenseFusion(df_estimator, img, points, choose, obj)
    bingham = BinghamInterpolation(torch.Tensor(max_q).unsqueeze(0).cuda(), 
                                   torch.ones(1).cuda(), 
                                   sigma=torch.Tensor(sigmas_single[obj.item()]).cuda())
    return bingham, max_q, max_t
    
def bing_mixed(img, points, choose, obj):
    pred_q, pred_t, pred_c, _ = evaluateDenseFusion(df_estimator, img, points, choose, obj,
                                                    return_all = True)
    how_max, which_max = torch.max(pred_c, 1)
    max_q = to_np(pred_q[which_max.item()])
    max_t = to_np(pred_t[which_max.item()])
    bingham_mixture = BinghamInterpolation(pred_q.cuda(), pred_c.flatten().cuda(), 
                                           sigma=torch.Tensor(sigmas_mixture[obj.item()]).cuda())
    
    return bingham_mixture, max_q, max_t

def bing_droput(img, points, choose, obj):
    pred_q, pred_t, pred_c, _ = evaluateDenseFusion(df_dropout_estimator, img, points, choose, obj,
                                                    return_all = True)
    how_max, which_max = torch.max(pred_c, 1)
    max_q = to_np(pred_q[which_max.item()])
    max_t = to_np(pred_t[which_max.item()])
    bingham_mixture = BinghamInterpolation(pred_q.cuda(), pred_c.flatten().cuda(), 
                                           sigma=torch.Tensor(sigmas_mixture[obj.item()]).cuda())
    
    return bingham_mixture, max_q, max_t

lik_funcs = {'hist_reg_global':hist_reg_global,
             #'hist_reg_idv_global':hist_reg_idv_global,
             'hist_comp_global':hist_comp_global,
             #'hist_comp_global_prop':hist_comp_global_prop,
             #'hist_comp_global_funnel':hist_comp_global_funnel,
             #'hist_comp_local':hist_comp_local,
             #'hist_conf':hist_conf,
             #'bing_fixed':bing_fixed,
             #'bing_mixed':bing_mixed,
             #'bing_iso':bing_iso,
             }

In [52]:
func(img, points, choose, obj+1)

(array([6.0243390e-38, 9.8212585e-18, 6.5214782e-21, ..., 6.7959160e-15,
        3.8903968e-13, 4.3493822e-13], dtype=float32),
 array([-0.6172123 , -0.72789514,  0.24433826, -0.171804  ], dtype=float32),
 array([ 0.05819113, -0.07106589,  0.9622421 ], dtype=float32))

In [53]:
from se3_distributions.datasets.ycb_dataset import getYCBSymmeties
from object_pose_utils.utils.pose_processing import symmetricAngularDistance, meanShift
from object_pose_utils.utils.pose_error import add, adi
from quat_math import quaternion_matrix

tetra_interp = TetraInterpolation(2)

import pathlib

from tqdm import tqdm_notebook as tqdm

likelihood = {k:{obj:[] for obj in range(len(object_list))} for k in lik_funcs.keys()}

sym_angular_error = {k:{obj:[] for obj in range(len(object_list))} for k in lik_funcs.keys()}
add_error = {k:{obj:[] for obj in range(len(object_list))} for k in lik_funcs.keys()}
add_sym_error = {k:{obj:[] for obj in range(len(object_list))} for k in lik_funcs.keys()}

sym_angular_error_mode = {k:{obj:[] for obj in range(len(object_list))} for k in lik_funcs.keys()}
add_error_mode = {k:{obj:[] for obj in range(len(object_list))} for k in lik_funcs.keys()}
add_sym_error_mode = {k:{obj:[] for obj in range(len(object_list))} for k in lik_funcs.keys()}

bad_data = []
with torch.no_grad():
    for j, data in enumerate(tqdm(dataset)):
        obj, quat, trans, img, points, choose = data

        if(len(obj) == 0):
            bad_data.append(j)
            continue

        #sym_axis, sym_ang = getYCBSymmeties(obj.item())
        sym_axis, sym_ang = [],[]
        trans = to_np(trans)

        for k, func in lik_funcs.items():            
            lik_est, q_est, t_est = func(img, points, choose, obj+1)

            if(type(lik_est) in [BinghamInterpolation, isoLikelihood, duelLikelihood]):
                lik = lik_est(quat.unsqueeze(0).cuda()).item()
                q_mode = q_est
            else:
                if(type(lik_est) is torch.Tensor):
                    lik_est = to_np(lik_est.flatten())
                tetra_interp.setValues(lik_est)
                lik = tetra_interp.smooth(to_np(quat)).item()    
                q_mode = grid_vertices[np.argmax(lik_est)]
                #v_shift = meanShift(q_mode.cuda(), grid_vertices.cuda(), dist_est.cuda(),
                #                    sigma=np.pi/9, max_iter = 100)

            mat = quaternion_matrix(quat)

            err_ang = symmetricAngularDistance(torch.Tensor(q_est).unsqueeze(0), 
                                               quat.unsqueeze(0),
                                               sym_axis, sym_ang).item()*180/np.pi
            mat_est = quaternion_matrix(q_est)
            err_add = add(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                          model_clouds[obj.item()])
            err_adi = adi(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                          model_clouds[obj.item()])

            err_ang_mode = symmetricAngularDistance(torch.Tensor(q_mode).unsqueeze(0), 
                                                    quat.unsqueeze(0),
                                                    sym_axis, sym_ang).item()*180/np.pi

            mat_mode = quaternion_matrix(q_mode)
            err_add_mode = add(mat[:3,:3], trans, mat_mode[:3,:3], t_est, 
                               model_clouds[obj.item()])
            err_adi_mode = adi(mat[:3,:3], trans, mat_mode[:3,:3], t_est, 
                               model_clouds[obj.item()])

            likelihood[k][obj.item()].append(lik)            
            sym_angular_error[k][obj.item()].append(err_ang)
            add_error[k][obj.item()].append(err_add)
            add_sym_error[k][obj.item()].append(err_adi)

            sym_angular_error_mode[k][obj.item()].append(err_ang_mode)
            add_error_mode[k][obj.item()].append(err_add_mode)
            add_sym_error_mode[k][obj.item()].append(err_adi_mode)

    np.savez('iros_results/linemod_{}.npz'.format('_'.join(lik_funcs.keys())), 
             likelihood=likelihood,
             sym_angular_error=sym_angular_error,
             add_error=add_error,
             add_sym_error=add_sym_error,
             sym_angular_error_mode=sym_angular_error_mode,
             add_error_mode=add_error_mode,
             add_sym_error_mode=add_sym_error_mode,
            )

Exception on index 584: No points in mask
Exception on index 7699: No points in mask
Exception on index 7700: No points in mask


In [28]:
grid_features.keys()

dict_keys([1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15])

In [ ]:
print(makeTableEntries(likelihood, val_func=mean_log_clean, bold_func = max, individual = True))
#makeTable(sym_angular_error, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTable(add_error, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTable(add_sym_error, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTableEntries(sym_angular_error_mode, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTableEntries(add_error_mode, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')
#makeTableEntries(add_sym_error_mode, val_func=mean_abs, bold_func = min, individual = True)
#print('\n\n\n')



In [ ]:
print(makeTableEntries(likelihood, objs = non_sym_objs, val_func=mean_log_clean, bold_func = max, agg_title = 'Non-Symmetric'))
print(makeTableEntries(likelihood, objs = sym_objs, val_func=mean_log_clean, bold_func = max, agg_title = 'Symmetric'))
print(makeTableEntries(likelihood, val_func=mean_log_clean, bold_func = max))